# Introduction

Welcome to Adbot Challenge! The objective of this challenge is to accurately predict the number of clicks a client’s ad receives, one and two weeks into the future (in digital marketing, clicks refer to when someone views the advert and follows one of the hyperlinks in that advert). Not only that, the challenge also requires one to find the most significant features on each prediction. Therefore, the best solution will not only produce an accurate prediction, but also the interpretation.

# Loading Libraries

There are 10 libraries we are using in this notebook:

1. Pandas for storing dataframe and creating new features;
2. Polars strictly for preprocessing and wrangling datasets;
3. Numpy strictly for creating array and setting seed value;
4. Matplotlib for visualization;
5. Seaborn for visualization;
5. SHAP for interpretation;
6. Holidays for determining whether a date is an holiday or not;
7. Category Encoders for categorical encoding;
8. Scikit-Learn for building pipeline;
9. LightGBM for machine learning model.

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap

import holidays
from category_encoders import MEstimateEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

from lightgbm import LGBMRegressor

from pathlib import Path

seed = 42
np.random.seed(seed)
FILEDIR = Path('../..')

# Loading and Preprocessing Datasets

The dataset we have contains the observation for each ad that Adbot's client have. However, what we need is to find the total ad clicks that each client get for a single day. Therefore, we need to aggregate the train data based on client ID and date. The features that we use from the original dataset are

1. client ID,
2. date,
3. ad type,
4. currency,
5. call type,
6. call status
7. duration, and
8. display location.

We also don't have the test dataset for this challenge. However, we are given sample submission. This is helpful for us as we can wrangle the sample submission instead of building test dataset from scratch. One thing to note is that we are not given any features for test dataset other than date and ID, as those two are the only known features in the future. Therefore, we use forward filling to create features in dataset.

Note 1: Categorical features mode aggregation process is non-deterministic due to the possibility of a feature having more than one mode, and every run produces different order.
Note 2: `duration` is recognized as string by Polars. Ideally, we changed it to float. However, we will keep it as string because that was how the best submission was created.

In [ ]:
datetime_preprocessor = [
    pl.col('date').cast(pl.Date),
    pl.col('start_time').str.strip_chars_end().str.to_datetime('%Y-%m-%d %H:%M:%S').cast(pl.Datetime),
    pl.col('end_time').str.strip_chars_end().str.to_datetime('%Y-%m-%d %H:%M:%S').cast(pl.Datetime),
]

datetime_calculator = [
    pl.col('date').cast(pl.Float32).alias('date_int')
]

unaggregated_train = pl.scan_csv(FILEDIR / 'input' / 'Train.csv').\
    with_columns(datetime_preprocessor).\
    with_columns(datetime_calculator)

train = unaggregated_train.group_by('ID', 'date').agg(
    pl.col('date_int').median().cast(pl.Int16),
    pl.col('clicks').sum().cast(pl.Float32),
    pl.col(pl.String).mode().map_elements(lambda x: x[0], return_dtype = pl.String),
).sort(['date', 'ID'])

test = pl.scan_csv(FILEDIR / 'input' / 'SampleSubmission.csv').\
    with_columns(
        pl.col('ID').str.head(27),
        pl.col('ID').str.tail(10).str.to_datetime('%Y_%m_%d').cast(pl.Date).alias('date')
    ).with_columns(
        pl.col('date').cast(pl.Int16).alias('date_int'),
    ).with_columns(
        clicks = None
    ).with_columns(pl.col('clicks').cast(pl.Float32)).select(pl.col('ID', 'date', 'date_int', 'clicks')).join(
        train,
        on = ['ID', 'date', 'date_int', 'clicks'],
        how = 'left'
    )

full_dataset = pl.concat(
    [train, test], how = 'vertical_relaxed'
).sort(
    ['ID', 'date']
).with_columns(
    pl.exclude('clicks').forward_fill()
)

train = full_dataset.filter(~pl.col('clicks').is_null()).sort(['date', 'ID'])
test = full_dataset.filter(pl.col('clicks').is_null()).drop('clicks').collect().to_pandas()

In [ ]:
X = train.collect().to_pandas()
y = X.pop('clicks')

cat_features = list(X.select_dtypes('object'))

# Feature Engineering

We only create new feature: holiday. If a date is an holiday, it will have a value of 1. Otherwise, it will be 0. The feature engineering will be done in a function called `holiday_generator`, and we build Scikit-Learn wrapper with `FunctionTransformer` so we can implement it in Scikit-Learn pipeline.

Note: We only generate holidays in South Africa.

In [ ]:
def holiday_generator(df: pd.DataFrame) -> pd.DataFrame:
    years = list(df.date.dt.year.unique())
        
    df_copy = df.copy()
        
    holiday_map = holidays.country_holidays('ZA', years = years)
        
    df_copy['holiday_name'] = df_copy.date.map(holiday_map)
    df_copy['is_holiday'] = np.where(df_copy.holiday_name.notnull(), 1, 0)
    df_copy = df_copy.drop(['holiday_name', 'date'], axis = 1)
        
    return df_copy

HolidayGenerator = FunctionTransformer(holiday_generator)

# Model

We build a preprocessor pipeline consisting of `HolidayGenerator` to generate holiday indicator and `MEstimateEncoder` to encode categorical features. As for our machine learning model, we will use LightGBM with DART bootstrapping and GOSS data sampling strategy. From this, we preprocess our predictors in the train dataset and the test dataset.

Note: We could have built a single pipeline from preprocessing until predicting. However, due to ease of interpreting our model with SHAP, we chose to separate them.

In [ ]:
Preprocessor = make_pipeline(
    HolidayGenerator,
    MEstimateEncoder(cat_features)
)

Preprocessor.fit(X, y)
preprocessed_X = Preprocessor.transform(X)
preprocessed_test = Preprocessor.transform(test)

In [ ]:
model = LGBMRegressor(
    random_state = seed,
    verbose = -1,
    boosting_type = 'dart', 
    data_sample_strategy = 'goss'
)

model.fit(preprocessed_X, y)

# Predicting and Calculating SHAP Values

In order to interpret the result with SHAP, we build our explainer based on our fitted LightGBM and preprocessed features.

In [ ]:
explainer = shap.TreeExplainer(model, preprocessed_X)
test_explanation = explainer(preprocessed_test)
shap_value = explainer.shap_values(preprocessed_test)

predictions = model.predict(preprocessed_test)

# Creating Submission Files

We create submission file based on our predictions and most important features. For the purpose of this competition, we limit the list of most important features to 5 features. Top 5 features are determined by the biggest absolute value of its SHAP values.

Note: If we use less than 5 features, the least important features will be `NULL`.

In [ ]:
importance_columns = [f'feature_{i}' for i in range(1, shap_value.shape[1] + 1 if shap_value.shape[1] < 5 else 6)]

submission = pd.concat([
    pd.Series(test.ID + '_' + test.date.astype(str).str.replace('-', '_'), name = 'ID'),
    pd.Series(predictions, name = 'target'),
    pd.DataFrame((-1 * np.abs(shap_value)).argsort()[:, :5], columns = importance_columns)
], axis = 1)

for col in importance_columns:
    submission[col] = submission[col].apply(lambda x: preprocessed_X.columns[x])

if submission.shape[1] < 7:
    for i in range(submission.shape[1] - 1, 6):
        submission[f'feature_{i}'] = 'NULL'

In [ ]:
submission.to_csv(FILEDIR / 'output' / 'submission.csv', index = False)

# Sensitivity Analysis

In this section, we will see how much the top 5 important features change independently to improve the total ad clicks. We will obtain the SHAP values for the top 5 features and concat it with our submission dataframe for ease of use.

In [ ]:
top_5_values = pd.DataFrame(shap_value).apply(lambda row: pd.Series(sorted(row, key = lambda row: -1 * np.abs(row))), axis = 1).iloc[:, :5]
top_5_values.columns = [col + '_values' for col in list(submission.iloc[:, 2:])]
sensitivity_table = pd.concat([submission, top_5_values], axis = 1)
sensitivity_table

Below, is an example of visualization of SHAP value for a single observation.

In [ ]:
plt.figure(figsize = (20, 15), dpi = 300)
plt.title("SHAP Value for a Client in a Day", size = 15, weight = 'bold')
shap.plots.waterfall(test_explanation[0])
plt.show()

There are four plots shown below. The first two measures the impact of each feature on each target, while the last two shows the variance on the level of importance on the features.

In [ ]:
plt.figure(figsize = (20, 15), dpi = 300)
plt.title('Impact Measurement on Each Feature per Observation', size = 13, weight = 'bold')
shap.plots.beeswarm(test_explanation)
plt.show()

In [ ]:
plt.figure(figsize = (20, 15), dpi = 300)
plt.title('Impact Measurement on Each Feature per Clustered Observation', size = 13, weight = 'bold')
shap.plots.heatmap(test_explanation)
plt.show()

In [ ]:
plt.figure(figsize = (15, 20), dpi = 300)
importance_counter = submission\
    .iloc[:, 2:]\
    .melt()\
    .rename(
        {'variable' : 'importance_level', 'value' : 'features'},
        axis = 1
    )
importance_counter['importance_level'] = importance_counter['importance_level']\
    .str\
    .replace('feature_1', '1st')\
    .replace('feature_2', '2nd')\
    .replace('feature_3', '3rd')\
    .replace('feature_4', '4th')\
    .replace('feature_5', '5th')
sns.countplot(
    data = importance_counter, 
    y = 'features', 
    hue = 'importance_level'
)
plt.title('Importance Level Count per Feature', size = 25, weight = 'bold')
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 2, figsize = (16, 25), dpi = 300)

for i, column in enumerate(importance_columns):

    ax[i][0].pie(
        submission[column].value_counts(), 
        shadow = True, 
        explode = [.1 for i in range(submission[column].nunique())], 
        autopct = '%1.f%%',
        textprops = {'size' : 14, 'color' : 'white'},
    )

    sns.countplot(
        data = submission, 
        y = column, 
        ax = ax[i][1], 
        order = submission[column].value_counts().index
    )
    ax[i][1].yaxis.label.set_size(20)
    plt.yticks(fontsize = 12)
    ax[i][1].set_xlabel('Count', fontsize = 15)
    ax[i][1].set_ylabel(f'Important Feature {i+1}', fontsize = 15)
    plt.xticks(fontsize = 12)

fig.suptitle('Feature Count per Importance Level\n\n', fontsize = 25, fontweight = 'bold')
plt.tight_layout()